In [1]:
"""
In this notebook i apply simple LSTM architecture to a classification problem.

"""
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
import jovian
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

In [2]:
#loading the data
reviews = pd.read_csv("D:/data7/Womens Clothing E-Commerce Reviews.csv")


In [3]:
print(reviews.shape)
reviews.head()

(23486, 11)


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [4]:
#check nulls
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [5]:
#fill nulls
cols = ['Title','Review Text']
for  col in cols:
    reviews[col] = reviews[col].fillna("")

In [6]:
#check nulls again
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    23486 non-null  object
 4   Review Text              23486 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [7]:
reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']
reviews.head(5)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,review
0,0,767,33,,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,Absolutely wonderful - silky and sexy and com...
1,1,1080,34,,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,Love this dress! it's sooo pretty. i happen...
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,Some major design flaws I had such high hopes ...
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"My favorite buy! I love, love, love this jumps..."
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,Flattering shirt This shirt is very flattering...


In [46]:
#get a slice of data
review_rating = reviews[['review', 'Rating']]

In [47]:
review_rating.head(3)

,review,Rating
0,Absolutely wonderful - silky and sexy and com...,4
1,Love this dress! it's sooo pretty. i happen...,5
2,Some major design flaws I had such high hopes ...,3


In [48]:
#normalize rating beginning index from 0
review_rating['Rating'] = review_rating['Rating'].apply(lambda x: x - 1)

C:\Users\mcelikkaya\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [49]:
review_rating.head(3)

,review,Rating
0,Absolutely wonderful - silky and sexy and com...,3
1,Love this dress! it's sooo pretty. i happen...,4
2,Some major design flaws I had such high hopes ...,2


In [14]:
#tokenization
#tok = spacy.load('en')
tok = spacy.load("en_core_web_sm")

In [15]:

def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [16]:
#count number of occurences of each word
counts = Counter()
for index, row in reviews.iterrows():
    counts.update(tokenize(row['review']))

In [17]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
deleted_words = []
for word in list(counts):
    if counts[word] < 2:
        deleted_words.append(word)
        del counts[word]
print("num_words after:",len(counts.keys()))
print("deleted_words ",deleted_words[0:10])

num_words before: 14138
num_words after: 8263
deleted_words  ['narrowing', 'cagrcoal', 'aded', 'mintue', 'darkler', 'mathced', 'everythiing', 'msallet', 'jkeep', 'matvehd']


In [18]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [19]:
print( list( vocab2index.keys() )[0:10] )
print( list( vocab2index.values() )[0:10] )

['', 'UNK', ' ', 'absolutely', 'wonderful', '  ', 'silky', 'and', 'sexy', 'comfortable']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [20]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    #generate an array of N zeros
    encoded = np.zeros(N, dtype=int)
    #find indexes of words for encoding
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    
    length = min(N, len(enc1))
    #copy part of encoding into arrays of 0
    encoded[:length] = enc1[:length]
    return encoded, length

In [21]:
print( encode_sentence("test sentence",vocab2index) )
#xxx and yy are not embeddings so they must be 1
print( encode_sentence("xxx yy",vocab2index) )

(array([5068, 6802,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0]), 2)
(array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0]), 2)


In [22]:
review_rating['encoded'] = review_rating['review'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
review_rating.head()

C:\Users\mcelikkaya\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,review,Rating,review_len,encoded
0,Absolutely wonderful - silky and sexy and com...,3,8,"[[2, 3, 4, 5, 6, 7, 8, 7, 9, 0, 0, 0, 0, 0, 0,..."
1,Love this dress! it's sooo pretty. i happen...,4,62,"[[2, 10, 11, 12, 5, 13, 14, 15, 16, 5, 17, 18,..."
2,Some major design flaws I had such high hopes ...,2,102,"[[54, 55, 56, 57, 17, 58, 59, 60, 61, 62, 11, ..."
3,"My favorite buy! I love, love, love this jumps...",4,25,"[[68, 109, 110, 2, 17, 10, 2, 10, 2, 10, 11, 1..."
4,Flattering shirt This shirt is very flattering...,4,38,"[[122, 123, 11, 123, 52, 92, 122, 19, 124, 125..."


In [23]:
#check how balanced the dataset is
Counter(review_rating['Rating'])

Counter({3: 5077, 4: 13131, 2: 2871, 1: 1565, 0: 842})

In [24]:
X = list(review_rating['encoded'])
y = list(review_rating['Rating'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [25]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [34]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [35]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [36]:
"""
A utility class to log all the sub steps of a
pytorch calculation.
Usage is sending data with label.
If data send is torch tensor, 
"""
class StepLogger():
    def __init__(self,capacity):
        self.tensor_datas = {}        
        self.capacity = capacity
        self.added_labels = []
        
    
    def add_info(self,tensor_data,tensor_label):
        if tensor_label not in self.added_labels:
            self.added_labels.append( tensor_label )
        
        if tensor_label in self.tensor_datas.keys():
            current_arr = self.tensor_datas.get(tensor_label)
            if len(current_arr) < self.capacity:
                current_arr = self.tensor_datas.get(tensor_label, [])
                current_arr.append(tensor_data)
        else:
            self.tensor_datas[tensor_label] = [tensor_data]
    
    def get_default_summary(self,show_data=False,summary_count=1):
        self.get_summary(self.added_labels,show_data)
        
    def get_summary(self,labels,show_data=False,summary_count=1):
        count = 0
        for i in range(summary_count):
            for l in labels:
                label_data = self.tensor_datas.get(l)[count]
                print(l," :")
                if torch.is_tensor(label_data):
                    print( list(label_data.size() ) )
                if not show_data and not torch.is_tensor(label_data):
                    print(label_data)
                if show_data:    
                    print(label_data)

In [37]:
def train_model_regr(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.float()
            step_logger.add_info(x,"train x")
            step_logger.add_info(y,"train y")
            y_pred = model(x, l)
            step_logger.add_info(y_pred,"train y_pred")
            
            optimizer.zero_grad()
            loss = F.mse_loss(y_pred, y.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss = validation_metrics_regr(model, val_dl)
        if i % 5 == 1:
            print("train mse %.3f val rmse %.3f" % (sum_loss/total, val_loss))

def validation_metrics_regr (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.float()
        y_hat = model(x, l)
        loss = np.sqrt(F.mse_loss(y_hat, y.unsqueeze(-1)).item())
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total

In [38]:
class LSTM_regr(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        step_logger.add_info(x,"forward x")
        step_logger.add_info(l,"forward l")
        embedded = self.embeddings(x)
        step_logger.add_info(embedded,"forward embedded")
        dropped = self.dropout(embedded)
        step_logger.add_info(dropped,"forward dropped")
        lstm_out, (ht, ct) = self.lstm(dropped)
        step_logger.add_info(lstm_out,"forward lstm_out")
        step_logger.add_info(ht,"forward ht")
        step_logger.add_info(ct,"forward ct")
        step_logger.add_info(ht[-1],"forward ht[-1]")
        
        return self.linear(ht[-1])

In [39]:
model

LSTM_regr(
  (embeddings): Embedding(8265, 50, padding_idx=0)
  (lstm): LSTM(50, 50, batch_first=True)
  (linear): Linear(in_features=50, out_features=1, bias=True)
  (dropout): Dropout(p=0.2)
)

In [42]:
step_logger =  StepLogger(2)
model =  LSTM_regr(vocab_size, 50, 50)

In [43]:
train_model_regr(model, epochs=10, lr=0.05)

train mse 1.534 val rmse 1.214
train mse 1.208 val rmse 1.132


In [44]:
step_logger.get_default_summary(False)

train x  :
[5000, 70]
train y  :
[5000]
forward x  :
[5000, 70]
forward l  :
[5000]
forward embedded  :
[5000, 70, 50]
forward dropped  :
[5000, 70, 50]
forward lstm_out  :
[5000, 70, 50]
forward ht  :
[1, 5000, 50]
forward ct  :
[1, 5000, 50]
forward ht[-1]  :
[5000, 50]
train y_pred  :
[5000, 1]


In [45]:
step_logger.get_default_summary(True)

train x  :
[5000, 70]
tensor([[3050,  537,   11,  ...,    0,    0,    0],
        [  92,   16,    2,  ...,   33,    2,    7],
        [ 114, 1122,  111,  ..., 1244,   21,   37],
        ...,
        [ 574,   71,  237,  ...,   74,   58,   19],
        [   2,    0,    0,  ...,    0,    0,    0],
        [  10,   71,  248,  ...,   29,   30,  305]])
train y  :
[5000]
tensor([0., 3., 4.,  ..., 4., 1., 2.])
forward x  :
[5000, 70]
tensor([[3050,  537,   11,  ...,    0,    0,    0],
        [  92,   16,    2,  ...,   33,    2,    7],
        [ 114, 1122,  111,  ..., 1244,   21,   37],
        ...,
        [ 574,   71,  237,  ...,   74,   58,   19],
        [   2,    0,    0,  ...,    0,    0,    0],
        [  10,   71,  248,  ...,   29,   30,  305]])
forward l  :
[5000]
tensor([30, 70, 70,  ..., 70,  1, 70])
forward embedded  :
[5000, 70, 50]
tensor([[[ 1.0936,  0.8604,  0.1514,  ..., -1.6385, -0.6484,  0.4473],
         [-1.2094, -1.1021, -0.7117,  ..., -2.4438,  0.1369, -0.3387],
         